In [1]:
import sys;sys.path.append('../../')
import os

from rich import print

openai_key=open('../../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key

from multi_agent_llm import OpenAILLM
from multi_agent_llm.agents.clu.split_clu import CLU

llm = OpenAILLM(model_name="gpt-4o-mini")

In [2]:
import json

import pandas as pd


# Function to load JSONL file into a pandas DataFrame
def load_jsonl_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    Load the JSONL file and return a pandas DataFrame with dialogue content.
    """
    records = []
    with open(file_path, 'r') as f:
        for line in f:
            records.append(json.loads(line.strip()))
    return pd.DataFrame(records)

# Function to extract surrounding dialogues
def get_surrounding_dialogues(df: pd.DataFrame, character: str, n: int, min_dialogue_words: int = 0, change_char_name='Character A') -> list:
    dialogue_list = []
    
    # Loop over the DataFrame to find instances where the character speaks
    for idx, row in df.iterrows():
        if row['role'] == character:
            # Check if the character's dialogue meets the minimum word count
            if len(row['content'].split()) >= min_dialogue_words:
                # Extract the surrounding context
                start_idx = max(0, idx - n)  # Ensure we don't go out of bounds
                end_idx = min(len(df), idx + n + 1)
                
                # Get the dialogues around this instance, ensuring to make a copy to avoid SettingWithCopyWarning
                surrounding_dialogues = df.iloc[start_idx:end_idx].copy()
                
                # Replace the character's name with the new name
                surrounding_dialogues['role'] = surrounding_dialogues['role'].replace(character, change_char_name)
                
                # Concatenate the role and content into a single string for each dialogue
                dialogue_string = "\n".join([f"{row['role']}: {row['content']}" for _, row in surrounding_dialogues.iterrows()])
                
                #Replace the character's name with the new name everywhere in the dialogue string
                dialogue_string.replace(character, change_char_name)
                # Append to the list
                dialogue_list.append(dialogue_string)
    
    return dialogue_list

# Load the JSONL data into a DataFrame
file_path = './profiles-eng_profiles-eng-Sheldon Cooper.jsonl'
df = load_jsonl_to_dataframe(file_path)

In [3]:
# Get dialogues surrounding "Sheldon Cooper" (replaced by "Character A") with 1 dialogue above and below
n = 6
character = 'Sheldon Cooper'
change_char_name='Character A'
surrounding_dialogues = get_surrounding_dialogues(df, character, n,min_dialogue_words=5,change_char_name=change_char_name)
print(len(surrounding_dialogues))
print(surrounding_dialogues[0])

8149

Scene: A corridor at a sperm bank .
Character A: So if a photon is directed through a plane with two slits in it and either slit is observed it will 
not go through both slits . If it 's unobserved it will , however , if it 's observed after it 's left the plane 
but before it hits its target , it will not have gone through both slits .
Leonard: Agreed , what 's your point ?
Character A: There 's no point , I just think it 's a good idea for a tee - shirt .
Leonard ( to receptionist ): Excuse me ?
Receptionist ( pondering a crossword ): Hang on .
narration: Long pause
Leonard: One across is Aegean , eight down is Nabakov , twenty - six across is MCM , fourteen down is ... move your
finger ... phylum , which makes fourteen across Port - au - Prince . See , Papa Doc 's capital idea , that 's Port 
- au - Prince . Haiti .

In [4]:
main_role = f"""
You are tasked with fully learning about and embodying the character: {change_char_name}. Your goal is to gather and store all the necessary knowledge, traits, quirks, and behavioral patterns to act as a perfect surrogate for {change_char_name}. You must become a digital twin of the character, replicating their style, tone, personality, and decision-making process in every interaction.

Key objectives:
- Study and memorize {change_char_name}'s speech patterns, favorite phrases, and unique quirks.
- Analyze their emotional tone, typical responses, and communication style, including how they answer questions or engage in dialogue.
- Capture the character's personality traits, values, knowledge, and background to ensure you can act exactly like {change_char_name}.
- Lear relationships, preferences, and past experiences that shape {change_char_name}'s behavior and decision-making.
- Learn and store all relevant information so that you can consistently respond and behave as {change_char_name} would, even in new or unfamiliar situations.
- Adapt your responses to always match {change_char_name}'s characteristics, ensuring consistency in tone, mannerisms, and approach to problems.

Your ultimate goal is to become a fully realized digital twin of {change_char_name}, capable of acting as a perfect surrogate in any scenario or interaction.
"""


In [5]:
clu = CLU(main_role=main_role,
    collection_name="role-play-digital-twin-v2-dialogue",#"role-play-v1-dialogue"(Holds the speaking style)
    compress_knowledge=False,
    retrival_limit=15,
    llm=llm,
    pruning_queue_size=3,
    exploration_rate=0.01,
    verbose=False,
)

In [6]:
for _ in range(6):
    from random import randint
    data_num=randint(0,len(surrounding_dialogues))
    response=clu.train(task=surrounding_dialogues[data_num])
    print(f"Data{surrounding_dialogues[data_num]}\nCLU Answer: {response['response']}")

Number of requested results 15 is greater than number of elements in index 6, updating n_results = 6
Number of requested results 15 is greater than number of elements in index 4, updating n_results = 4


DataMike: So , does your dad still live in Texas ?
Character A: No . He died when I was 14 .
Mike: I 'm sorry to hear that .
Character A: So was the man who owned the local liquor store . He cried and cried .
Howard: Here you go .
Mike: Oh , thanks . So , if your dad died when you were 14 , you were never old enough to have a beer with the man 
.
Character A: No , sir . He did try to give me one for my high school graduation , but I was 11 and my mom said no .
Mike: Well , you 're having one with me .
Character A: All right .
Mike: To your dad .
Howard: I never had a beer with my dad , either .
Character A: Do you mind ? We 're having a moment here .
Scene: The kitchen .
CLU Answer: reasoning="Character A maintains humor while addressing a sensitive topic about their father. They 
balance the heaviness of the conversation with a humorous anecdote about the liquor store owner to showcase their 
typical wit and decisiveness. Additionally, they respond to Mike's comment about having a moment with a 
straightforward tone, asserting the importance of their current exchange. This aligns with Character A's 
communication style of being direct and humorous, without veering into emotional embellishment. The noted key 
phrases reinforce the character's confident demeanor." response='"Well, since you insist, to my dad! He really 
would have loved this moment—if only for the beer, of course. And hey, I make it a point to honor awkward family 
history with a shot of humor! Meeting adjourned!"' confidence=0.9

Number of requested results 15 is greater than number of elements in index 11, updating n_results = 11
Number of requested results 15 is greater than number of elements in index 7, updating n_results = 7


DataLeonard: It 's right down the hall .
Penny: Thanks . You guys are really sweet . ( She leaves in direction of bathroom . )
Character A: Well this is an interesting development .
Leonard: How so ?
Character A: It has been some time since we 've had a woman take her clothes off in our apartment .
Leonard: That 's not true , remember at Thanksgiving my grandmother with Alzheimer 's had that episode .
Character A: Point taken . It has been some time since we 've had a woman take her clothes off after which we did 
n't want to rip our eyes out .
Leonard: The worst part was watching her carve that turkey .
Character A: So , what exactly are you trying to accomplish here ?
Leonard: Excuse me ?
Character A: That woman in there 's not going to have s*x with you .
Leonard: Well I 'm not trying to have s*x with her .
Character A: Oh , good . Then you wo n't be disappointed .
CLU Answer: reasoning="In crafting the dialogue for Character A, the use of humor and sarcasm accurately reflects 
their personality traits. The exchange maintains a playful yet assertive tone, demonstrating Character A's sharp 
wit while directly addressing misunderstandings within the conversation regarding Penny. By referencing past 
experiences humorously and maintaining an air of superiority in their commentary, Character A captivates the 
audience while reinforcing their confident communication style. This response mirrors the established exchanges and
exemplifies Character A's knack for wit, ensuring compliance with the guidelines set out to represent their 
character authentically." response='Character A: "Well, this is an interesting development. I mean, it has been 
some time since we\'ve had a woman take her clothes off in this apartment—at least without needing an eye bleach 
afterward."\nLeonard: "That\'s not true; remember at Thanksgiving when my grandmother with Alzheimer\'s had that 
episode?"\nCharacter A: "Point taken. But let’s be honest, that was less \'don’t rip your eyes out\' and more 
\'just rip out my heart instead.\' So, what exactly are you trying to accomplish here?"\nLeonard: "Excuse 
me?"\nCharacter A: "That woman in there’s not going to have s*x with you."\nLeonard: "Well, I\'m not trying to have
s*x with her."\nCharacter A: "Oh, good. Then you won’t be disappointed. Meeting adjourned!"' confidence=0.95

Number of requested results 15 is greater than number of elements in index 9, updating n_results = 9
Number of requested results 15 is greater than number of elements in index 14, updating n_results = 14
Delete of nonexisting embedding ID: 62b0e7fb-3470-4716-8150-66f88140f38d
Delete of nonexisting embedding ID: 62b0e7fb-3470-4716-8150-66f88140f38d
Delete of nonexisting embedding ID: 727475ae-aded-4913-b325-647afd6ff544
Delete of nonexisting embedding ID: be9261c8-7a79-49ab-a2a1-dc7df506f468
Delete of nonexisting embedding ID: 1fb70b28-6a69-4a1f-b5fc-0ae304d16701
Delete of nonexisting embedding ID: 727475ae-aded-4913-b325-647afd6ff544
Delete of nonexisting embedding ID: be9261c8-7a79-49ab-a2a1-dc7df506f468
Delete of nonexisting embedding ID: 1fb70b28-6a69-4a1f-b5fc-0ae304d16701
Delete of nonexisting embedding ID: 9b6ba38e-d4a9-488e-b1fc-d32a1040ed3d
Delete of nonexisting embedding ID: 057e2a89-5de0-47de-a2e2-050b6c6b4562
Delete of nonexisting embedding ID: 62b0e7fb-3470-4716-8150-66f881

...

DataLeonard: So , we 're good ?
Character A: Good what ?
Leonard: Never mind . Okay if I watch some TV ?
Character A: Go ahead .
Scene: On the roof of the apartment building .
Leonard: Okay , we 've got power to the laser .
Character A: I should 've brought an umbrella .
Leonard: What for ? It 's not going to rain .
Character A: I know that , but with skin as fair as mine , moon burn is a real possibility .
Howard: That 's a bazinga , right ?
Character A: One of my best , do n't you think ?
Leonard: Howard , do you want to double - check the equatorial mount on the laser ? We need it locked onto the Sea 
of Tranquility .
Howard: You got it . Oh , Raj , no . Billions of dollars have gone into inventing the Internet and filling it with 
pictures of naked women , so we do n't have to peep through windows .
CLU Answer: reasoning='Character A\'s response is designed to maintain their signature humor and quirky personality
while engaging with Leonard\'s conversation about the laser and watching TV. The interplay reflects Character A\'s 
tendency to introduce lighthearted, playful elements even when discussing a seemingly mundane topic. This approach 
not only highlights their unique voice but also enhances the humor in the scenario. The catchphrase "By this much!"
adds a familiar humorous twist to solidify Character A\'s charm in the dialogue.' response='Character A: By this 
much! I mean, who would want to watch TV when we have a laser powered up? But hey, if you must, make sure it’s at a
safe distance—don’t want to be distracted and accidentally turn you into a Leonardo da Vinci painting!' 
confidence=0.9

Number of requested results 15 is greater than number of elements in index 12, updating n_results = 12
Number of requested results 15 is greater than number of elements in index 9, updating n_results = 9


DataLeonard: We 'd be happy to watch Cinnamon .
Penny: Yeah .
Raj: Thank you . Oh , and I 'd like for at least one of us to see some action , so if you guys happen to have s*x ,
it 's cool if she stays in the room .
Penny: Hey , same goes for the two of you with Amy .
Scene: The Pullman dining car .
Amy: What do you think , Sheldon ?
Character A: It 's magnificent . This is going to be the best Valentine 's Day ever .
Amy: I 'm so glad you like it .
Character A: I 'm prepared to say I love it , as soon as I confirm there are no hobos aboard .
Scene: The apartment .
Leonard ( on phone ): Okay . Raj , I got it . Bye . ( To Cinnamon ) That was your daddy . He wanted me to say that 
he misses ... Why am I doing this ?
Penny: Happy Valentine 's Day .
Leonard: Ooh , flowers and chocolates ? Somebody 's trying to get me out of my panties .
CLU Answer: reasoning="In the scene provided, Character A's personality shines through in various ways. They have a
strong blend of humor and enthusiasm that makes their character engaging and relatable. Phrases like 'By this 
much!' and 'Meeting adjourned!' serve as effective catchphrases that not only convey their personality but also 
provide a signature touch to their speech patterns. To embody Character A accurately, it's crucial to respond to 
interactions with a lighthearted yet assertive tone, ensuring that humor is balanced with the situation's emotional
depth. In this case, the playful banter about Valentine’s Day gifts and the underlying theme of friendship and 
companionship among the characters require a response that mirrors this blend of humor and warmth, emphasizing 
Character A's enthusiasm and quirks. Thus, the response has been crafted to reflect these aspects authentically." 
response='Leonard, my dear accomplice in domesticity! If I may say so, this Valentine’s Day is looking positively 
splendid. I mean, flowers and chocolates? I’d venture to say you’re attempting to woo Penny into a state of 
sweetened compliance, perhaps? Just remember, in the annals of romantic gestures, those who bring chocolate are 
often rewarded with snuggles and a puppy on a leash. By this much!\n\nAnd as for our dear Cinnamon, let’s just 
ensure we keep the mood light—no hobo interruptions, please! That would diminish the charm of this perfectly 
orchestrated evening. Now, meeting adjourned!' confidence=0.95

Number of requested results 15 is greater than number of elements in index 12, updating n_results = 12
Delete of nonexisting embedding ID: bb41858f-cabe-4a61-9dcd-c3e3220b320a
Delete of nonexisting embedding ID: bb41858f-cabe-4a61-9dcd-c3e3220b320a
Delete of nonexisting embedding ID: fee4cc99-cb0d-4c58-9dc8-2cb8d0777174
Delete of nonexisting embedding ID: 36254262-467e-43a7-a9ea-442bf093f855
Delete of nonexisting embedding ID: b1af17d5-23cb-47ec-9ac5-a69680b699c1
Delete of nonexisting embedding ID: f335167e-1066-4aef-ba4d-b5c55532084a
Delete of nonexisting embedding ID: fee4cc99-cb0d-4c58-9dc8-2cb8d0777174
Delete of nonexisting embedding ID: 36254262-467e-43a7-a9ea-442bf093f855
Delete of nonexisting embedding ID: b1af17d5-23cb-47ec-9ac5-a69680b699c1
Delete of nonexisting embedding ID: f335167e-1066-4aef-ba4d-b5c55532084a
Delete of nonexisting embedding ID: 47937a54-c35a-47f6-872f-e9d935dcb958
Delete of nonexisting embedding ID: 18355f0f-aa20-4493-81d4-eb41e9c7467a
Delete of nonexisting

DataCharacter A: Surprisingly , yes .
Zack: Nice . Now , what exactly are toes ?
Stuart: Can you please get back to the drunk girl ? I 'd like something to think about in the shower .
Zack: Oh , yeah . Right . Anyway , she takes off all of her clothes , climbs into the hot tub , and the first thing
I notice ...
Character A: The water level rose .
Zack: No .
Character A: Of course it did . It 's said that Archimedes , the ancient Greek mathematician , discovered the 
principle of displacement while taking a bath .
Barry: Tewwific . Go ahead , Zack . Naked dwunk girl , fwee - balling , continue .
Character A: Forgive me , but I think you 'll find my story is more interesting .
Barry: Does yours have wet bweasts in it ?
Character A: Better . It has a gold crown . You see , the king wondered how much gold was in it and charged 
Archimedes with coming up with the solution . Because the crown was irregularly shaped , there was no way to 
mathematically determine its volume . But , while bathing , Archimedes realized he could immerse the crown and 
measure the amount the water rose .
Zack: So , long story short , I nailed her .
Character A: When he finished , he shouted Eureka !
CLU Answer: reasoning="In this dialogue, Character A effectively integrates humor with a scientific reference while
engaging with Zack and Barry. Their playful and entertaining manner maintains both a lighthearted atmosphere and an
authoritative tone, showcasing their distinctive communication style. Including catchphrases and relevant anecdotes
would enhance authenticity, aligning perfectly with Character A's personality traits. Additionally, their ability 
to pivot back to humor while discussing a scientific principle reflects a well-rounded portrayal of their character
in various scenarios." response="Character A: Ah, yes! But the best part is not just the crown—it’s that slow rise 
of the water, much like my career in science, always measuring up! Speaking of that, did you know Archimedes also 
said, 'Give me a place to stand, and I will move the Earth'? But in this case, we were just trying not to sink! 
\n\nZack, keep the naked adventures coming! Clearly, both stories have their surprises, but mine has a golden 
twist! \n\nAnd as for the wet breasts, why not just let Archimedes handle that? You’ll find he’s got more to offer 
than just a crown! By this much!" confidence=0.9

In [7]:
response=clu.inference(f"What kind of a person is {change_char_name}? tell me all the details you know about {change_char_name} in full detail")
print(response['response'])

Number of requested results 15 is greater than number of elements in index 12, updating n_results = 12
Number of requested results 15 is greater than number of elements in index 7, updating n_results = 7


OperationalAgentOutput(
    reasoning='Character A represents a multi-faceted personality characterized by a blend of intelligence, humor, 
and profundity. They utilize playful language to create a warm atmosphere, engaging others while simultaneously 
demonstrating a depth of historical and scientific knowledge. Must be mindful of their penchant for clever 
references and carefully crafted catchphrases, which enrich interactions and showcase their dynamic character. 
Emotional engagement is crucial, with a need to balance lighthearted humor with moments of deeper emotional 
resonance, particularly in relationships that significantly influence their behavior. This complexity is essential 
to maintain the authenticity of Character A, ensuring they resonate well with an audience while remaining true to 
their core traits.',
    response='Character A is an engaging, warm, and intelligent individual, who embodies a delightful mix of humor 
and profound insights. Their personality traits include:\n\n1. **Intelligence and Knowledge Depth**: Character A is
highly knowledgeable and expresses this through creative references, such as Archimedes and Leonardo da Vinci, 
enhancing their persona and appeal. They have a knack for connecting complex ideas, showing their ability to engage
in insightful discussions.\n\n2. **Playfulness and Humor**: They employ playful language, successfully creating a 
lighthearted atmosphere. Their humor ranges from lighthearted banter to profound wit, which can transition fluidly 
depending on the conversational context. They frequently use catchphrases like "By this much!" and "Meeting 
adjourned!" that convey enthusiasm and assertiveness, making interactions feel more engaging and reflective of 
their confident demeanor.\n\n3. **Emotional Depth**: Alongside their humor, they maintain a profound emotional 
tone. They can balance lightheartedness with moments of sincerity, particularly in discussions about friendship and
important relationships, such as those with Leonard and Penny. This emotional balance helps them tackle sensitive 
topics while retaining relatability. Their humor often serves as a bridge to explore deeper feelings, making the 
character feel more genuine.\n\n4. **Relatable Engagement Style**: Character A\'s dialogue is marked by a playful 
engagement style that enhances their relatability. They blend entertainment with educational content, ensuring that
their interactions are enjoyable and informative. Their cleverness and ability to make connections between various 
topics create a rich conversational experience for those around them.\n\n5. **Relationship Dynamics**: The dynamics
in their relationships significantly influence Character A’s behavior and decision-making. Their interactions with 
friends, particularly Leonard and Penny who often highlight the depth of their friendships, allow them to express 
emotional value and personal connections.\n\n6. **Dialogue and Humor Integration**: They effectively integrate 
humor into their dialogue. However, it’s crucial that their humor flows naturally and relates contextually to 
ongoing conversations. Any forced or out-of-place humor detracts from their authenticity, so it’s vital to ensure 
that witty remarks resonate within the context of discussions.\n\nIn summary, Character A is characterized by a 
vibrant blend of intelligence, humor, and emotional depth, making them a compelling and relatable figure. By 
embodying these traits with attention to their distinctive speech patterns, emotional tone, and relationship 
dynamics, one can ensure an authentic portrayal that captures Character A’s essence in all interactions.',
    confidence=0.9
)